## Domo arigato, Mr. Roboto

In [2]:
from pathlib import Path
import numpy as np
import re

In [3]:
pattern = 'Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore.'
pattern +=' Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.'

pattern = re.compile(pattern)

In [4]:
blueprints = [[int(num) for num in re.match(pattern, blueprint).groups()] 
              for blueprint in Path('blueprints_test.txt').read_text().split('\n')]

In [38]:
class Inventory():
    
    def __init__(self, blueprint, ore_robo = 0, clay_robo = 0, obs_robo = 0, 
                 geo_robo = 0, ore = 0, clay = 0, obs = 0, geo = 0):
        
        self.blueprint = blueprint
        
        self.ore_robo_cost = blueprint[1]
        self.clay_robo_cost = blueprint[2]
        self.obsidian_robo_cost = blueprint[3:5]
        self.geode_robo_cost = blueprint[5:]
        
        
        
        self.max_ore_cost = max(map(blueprints[0].__getitem__, [1, 2, 3, 5]))
        
        
        self.ore_robo = ore_robo
        self.clay_robo = clay_robo
        self.obs_robo = obs_robo
        self.geo_robo = geo_robo
        
        self.ore = ore
        self.clay = clay
        self.obs = obs
        self.geo = geo
        
    def mine(self):
        
        return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo, self.geo_robo, 
                         self.ore + self.ore_robo, self.clay + self.clay_robo, self.obs + self.obs_robo, 
                         self.geo + self.geo_robo)
        
    def build_options(self):
        options = []
        
        if self.ore >= self.ore_robo_cost and self.ore_robo < self.max_ore_cost:
            options += ['ore']
        if self.ore >= self.clay_robo_cost and self.clay_robo < self.obsidian_robo_cost[1]:
            options += ['clay']
            
        if self.ore >= self.obsidian_robo_cost[0] and self.clay >= self.obsidian_robo_cost[1] and self.obs_robo < self.geode_robo_cost[1]:
            options += ['obsidian']
            
        if self.ore >= self.geode_robo_cost[0] and self.obs >= self.geode_robo_cost[1]:
            options += ['geode']
            
        # Need to implement strategy to not wait
            
        return options
    
    def build(self, robo):
        
        if robo == 'ore':
            return Inventory(self.blueprint, self.ore_robo+1, self.clay_robo, self.obs_robo, 
                             self.geo_robo, self.ore - self.ore_robo_cost, self.clay, self.obs, self.geo)
        elif robo == 'clay':
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo+1, self.obs_robo, 
                             self.geo_robo, self.ore - self.clay_robo_cost, self.clay, self.obs, self.geo)
        elif robo == 'obsidian':
            ore_cost, clay_cost = self.obsidian_robo_cost
            
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo+1, 
                             self.geo_robo, self.ore - ore_cost, self.clay - clay_cost, self.obs, self.geo)
        elif robo == 'geode':
            ore_cost, obs_cost = self.geode_robo_cost
            
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo, 
                             self.geo_robo+1, self.ore - ore_cost, self.clay, self.obs - obs_cost, self.geo)
        
    def __repr__(self):
        resources = f'Ore: {self.ore}, Clay: {self.clay}, Obsidian: {self.obs}, Geode: {self.geo}'
        robots = f'Ore Robot: {self.ore_robo}, Clay Robot: {self.clay_robo}, '
        robots += f'Obsidian Robot: {self.obs_robo}, Geode Robot: {self.geo_robo}'
        
        return resources + '\n' + robots
    
    def __str__(self):
        return self.__repr__

In [46]:
def calculate_quality(blueprint, time_limit = 24):
    blueprint_num = blueprint[0]
    
    ore_robo_cost = blueprint[1]
    clay_robo_cost = blueprint[2]
    obsidian_robo_cost = blueprint[3:5]
    geode_robo_cost = blueprint[5:]
    
    
    current_time = 0
    
    start_inventory = Inventory(blueprint, ore_robo = 1)
    
    status = [start_inventory]
    
    while current_time < time_limit:
        new_inventories = []
        for inventory in status:
            build_options = inventory.build_options()
            
            inventory2 = inventory.mine()
            
            add_robo = [inventory2.build(robo) for robo in build_options]
            
            new_inventories += [inventory2] + add_robo
                
        status = new_inventories
        
        print(current_time, len(status))
        
        current_time += 1
    
    return status

In [27]:
test = [1, 2, 3, 5]

In [37]:
min(map(blueprints[0].__getitem__, test))

2

In [44]:
blueprints[0]

[1, 4, 2, 3, 14, 2, 7]

In [47]:
calculate_quality(blueprints[0], 24)

0 1
1 1
2 2
3 3
4 7
5 12
6 29
7 57
8 143
9 320
10 846
11 2202
12 6237
13 18267
14 56085
15 179588
16 591458
17 1997518


KeyboardInterrupt: 